# Wolaytta Word Embedding with Gensim

## Word embeddings 

A word embedding is an approach to provide a dense vector representation of words that capture something about their meaning. The vector space representation of the words provides a projection where words with similar meanings are locally clustered within the space. It is modern approach for representing text in natural language processing. Embedding algorithms like Word2Vec and GloVe are key to the state-of-the-art results achieved by neural network models on natural language processing problems like machine translation.

Wolaytta is poorly resourced and highly inflected language from Afro-asiatic language family. This is the first attempt to share a Wolaytta small text corpus, Word Embedding, and wolaytta wordsim100. The Wolaytta-word-embedding is a pre-trained distributed word representation, wordsim100 - provides human annotated scores of relatedness between term pairs collected form potential users which was used to evaluate word embedding model.

Things to see:

1. Punctuation

In [1]:
## Gensim is an open source Python library for natural language processing
import gensim, logging
import os
## from gensim.models import FastText
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.fasttext import FastText
from gensim.models import Word2Vec
import re

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/xgebt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
!pip3 install sentencepiece

In [4]:
import os, re
from nltk.tokenize import sent_tokenize
RAW_DATA_DIR = './data'
CLEAN_DATA_DIR = './clean'

In [5]:
import os, re
from nltk.tokenize import sent_tokenize
file_count = 1
saved_files = os.listdir(CLEAN_DATA_DIR)
if len(saved_files)>0:
    file_count = len(saved_files)
else:        
    for fname in os.listdir(RAW_DATA_DIR):
        try:
            if fname.endswith(".txt"):
                txt_file = open(os.path.join(RAW_DATA_DIR, fname),encoding='utf8').readlines()
                sen = []
                for s in txt_file:
                    sent = sent_tokenize(s)
                    for st in sent:
                        st = st.lower()
                        b = re.sub(r'\w+7\w+', '\'', st)
                        a = ('\n'+ b.replace('”', '\'\'').replace('’', '\'').replace('7', '\'').replace('_', '').replace('-', ''))
                        c = re.sub(r'\d+', 'NUM', a)
                        words = re.findall(r'\w+\'*\w+', c)
                        ss = ' '.join(x for x in words if x)
                        if ss not in sen:
                            sen.append(ss.encode("ascii", "ignore").decode())
                new_fname = f"{file_count}.txt"
                clean_file = open(os.path.join(CLEAN_DATA_DIR, new_fname),'w+',encoding='utf8')
                sen_per_line_test = ''
                for s in sen:
                    sen_per_line_test = sen_per_line_test +'\n'+s
                clean_file.write(sen_per_line_test)

                file_count+=1
        except:
            continue
f"{file_count} files created to {CLEAN_DATA_DIR}"

'2880 files created to ./clean'

In [6]:
if not os.path.exists('./subword-model/all_in_one.txt'):
    for fname in os.listdir(CLEAN_DATA_DIR):
        txt_file = open(os.path.join(CLEAN_DATA_DIR, fname),encoding='utf8').readlines()
        clean_file = open(os.path.join('./subword-model', 'all_in_one.txt'),'a+',encoding='utf8')
        for s in txt_file:
            clean_file.write(s)

In [7]:
#SentencePiece is an unsupervised text tokenizer and detokenizer mainly for Neural Network-based 
#text generation systems where the vocabulary size is predetermined prior to the neural model training. 
#SentencePiece implements subword units BPE and unigram language model.
#It treats the sentences just as sequences of Unicode characters. 
import sentencepiece as spm
spm.SentencePieceTrainer.Train('--input=' + os.path.join('./subword-model', 'all_in_one.txt') +
            ' --model_prefix=wol_sp --vocab_size=16000 --hard_vocab_limit=false')

#It is a unigram_model_trainer.cc(138) LOG(INFO) Making suffix array...
#unigram_model_trainer extracts frequent sub strings...
# Then save file at /Wolaytta_Word_Embedding.ipynb

In [8]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.Load("./subword-model/wol_sp.model")
sp.EncodeAsPieces("higgiyyappe")

['▁hi', 'ggi', 'yya', 'ppe']

In [9]:
#Open Wolaytta sentence list
file = open("./subword-model/all_in_one.txt", "r", encoding="utf8")
number_of_lines = 0
number_of_words = 0
number_of_characters = 0
for line in file:
    line = line.strip("\n")
    words = line.split()
    number_of_lines += 1
    number_of_words += len(words)
    number_of_characters += len(line)
file.close()

print("lines:", number_of_lines, "words:", number_of_words, "characters:", number_of_characters)

lines: 863006 words: 6310966 characters: 50965677


In [13]:
#List of tokenized list of sentences
#wordList = []
#for word in file.readlines():
#z = re.findall('\w+\'*\w+', word) 
    #print(z)
   # wordList.append(z)
#for a in wordList:
   # print(a)

In [14]:
print('Number of Tokens in text file :', len(wordList), "\n\n")

NameError: name 'wordList' is not defined

In [15]:
import logging
import os
from gensim.models.fasttext import FastText

EMBEDDING_DIR='./emb_models'
PREPROCESSED_DIR='./clean'
class WEConfig(object):
    """Training parameters"""
    window=5 #Maximum skip length window between words
    emb_dim=50 # Set size of word vectors
    emb_lr=0.05 #learning rate for SGD estimation.
    nepoach=20 #number of training epochs
    nthread=20 #number of training threads
    sample = 0 #Set threshold for occurrence of words. Those that appear with higher frequency in the training data will be randomly down-sampled
    negative = 15 #negative sampling is used with defined negative example
    hs = 0 #0 Use Hierarchical Softmax; default is 0 (not used)
    binary=0 # 0 means not saved as .bin. Change to 1 if allowed to binary format
    sg=1 # 0 means CBOW model is used. Change to 1 to use Skip-gram model
    iterate=10 # Run more training iterations
    minFreq=2 #This will discard words that appear less than minFreq times 
    WORD_VECTOR_CACHE=EMBEDDING_DIR+'wol_word_vectors_sts.npy'
    if sg==0:
      model_name='wol_fasttext_cbow_'+str(emb_dim)+'D'
    elif sg==1:
      model_name='wol_fasttext_sg_'+str(emb_dim)+'D'
    
class corpus_sentences(object):# accept sentence stored one per line in list of files inside defined directory
    def __init__(self, dirname, sub_word = True):
        self.dirname = dirname
        self.sub_word = sub_word
        
        if self.sub_word:
            self.sp=spm.SentencePieceProcessor()
            self.sp_model=self.sp.Load("./subword-model/wol_sp.model")
    
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname),encoding='utf8'):
                 if self.sub_word:
                    yield self.sp.EncodeAsPieces(line)
                 else:
                    yield line.split()

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)


In [16]:
# wol_model=load_wol_word_vectors(sub_word = True)

In [17]:
def train_w2v_model(sub_word = True):
    print('Loading Sentences with memory freindly iterator ...\n')
    sentences = corpus_sentences(PREPROCESSED_DIR, sub_word) # a memory-friendly iterator 
    if WEConfig.sg==0:
        model_type='CBOW'
    else:
        model_type='Skip-gram'
    if sub_word:
        model_name=f'subword_wol_W2V_{model_type}_{WEConfig.emb_dim}D'
    else:
        model_name=f'wol_W2V_{model_type}_{WEConfig.emb_dim}D'
    print('Training Sentence Piece Word2Vec '+model_type+' with '+str(WEConfig.emb_dim)+' dimension\n') 
    _model = Word2Vec(sentences, vector_size=WEConfig.emb_dim, window=WEConfig.window, 
                            min_count=WEConfig.minFreq, workers=WEConfig.nthread,sg=WEConfig.sg,
                            epochs=WEConfig.iterate,negative=WEConfig.negative,
                            hs=WEConfig.hs,sorted_vocab=1)  
    _model.build_vocab(sentences)
    #trim unneeded model memory = use (much) less RAM
    _model.init_sims(replace=True)
    
    #Saving model   
    model_path=os.path.join(EMBEDDING_DIR,model_name)
    _model.save(model_path)

    return _model

In [18]:
def train_FastText_model(sub_word = True):
    print('Loading Sentences with memory freindly iterator ...\n')
    sentences = corpus_sentences(PREPROCESSED_DIR, sub_word) # a memory-friendly iterator 
    if WEConfig.sg==0:
        model_type='CBOW'
    else:
        model_type='Skip-gram'
    if sub_word:
        model_name=f'subword_wol_FastText_{model_type}_{WEConfig.emb_dim}D'
    else:
        model_name=f'wol_FastText_{model_type}_{WEConfig.emb_dim}D'
    print('Training Sentence Piece Word2Vec '+model_type+' with '+str(WEConfig.emb_dim)+' dimension\n') 
    _model = FastText(sentences, vector_size=WEConfig.emb_dim, window=WEConfig.window, 
                            min_count=WEConfig.minFreq, workers=WEConfig.nthread,sg=WEConfig.sg,
                            epochs=WEConfig.iterate,negative=WEConfig.negative,
                            hs=WEConfig.hs,sorted_vocab=1)
    _model.build_vocab(sentences)

    #trim unneeded model memory = use (much) less RAM
    _model.init_sims(replace=True)
    
    #Saving model   
    model_path=os.path.join(EMBEDDING_DIR,model_name)
    _model.save(model_path)

    return _model

In [ ]:
train_FastText_model(sub_word = False)

2021-11-29 11:56:49,352 : INFO : collecting all words and their counts
2021-11-29 11:56:49,356 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-11-29 11:56:49,461 : INFO : PROGRESS: at sentence #10000, processed 94367 words, keeping 25860 word types
2021-11-29 11:56:49,485 : INFO : PROGRESS: at sentence #20000, processed 137135 words, keeping 34975 word types
2021-11-29 11:56:49,500 : INFO : PROGRESS: at sentence #30000, processed 157135 words, keeping 43701 word types
2021-11-29 11:56:49,513 : INFO : PROGRESS: at sentence #40000, processed 177135 words, keeping 52299 word types
2021-11-29 11:56:49,527 : INFO : PROGRESS: at sentence #50000, processed 197135 words, keeping 60914 word types
2021-11-29 11:56:49,540 : INFO : PROGRESS: at sentence #60000, processed 217135 words, keeping 69543 word types
2021-11-29 11:56:49,554 : INFO : PROGRESS: at sentence #70000, processed 237135 words, keeping 77508 word types


Loading Sentences with memory freindly iterator ...

Training Sentence Piece Word2Vec Skip-gram with 50 dimension



2021-11-29 11:56:49,568 : INFO : PROGRESS: at sentence #80000, processed 257135 words, keeping 84909 word types
2021-11-29 11:56:49,583 : INFO : PROGRESS: at sentence #90000, processed 277135 words, keeping 90915 word types
2021-11-29 11:56:49,598 : INFO : PROGRESS: at sentence #100000, processed 296511 words, keeping 95135 word types
2021-11-29 11:56:49,643 : INFO : PROGRESS: at sentence #110000, processed 398973 words, keeping 103853 word types
2021-11-29 11:56:49,682 : INFO : PROGRESS: at sentence #120000, processed 495786 words, keeping 109784 word types
2021-11-29 11:56:49,703 : INFO : PROGRESS: at sentence #130000, processed 541473 words, keeping 110742 word types
2021-11-29 11:56:49,713 : INFO : PROGRESS: at sentence #140000, processed 555554 words, keeping 110894 word types
2021-11-29 11:56:49,729 : INFO : PROGRESS: at sentence #150000, processed 582222 words, keeping 111943 word types
2021-11-29 11:56:49,765 : INFO : PROGRESS: at sentence #160000, processed 672357 words, keepi

2021-11-29 11:56:51,661 : INFO : PROGRESS: at sentence #800000, processed 5690369 words, keeping 166761 word types
2021-11-29 11:56:51,700 : INFO : PROGRESS: at sentence #810000, processed 5808672 words, keeping 166774 word types
2021-11-29 11:56:51,730 : INFO : PROGRESS: at sentence #820000, processed 5887884 words, keeping 167572 word types
2021-11-29 11:56:51,767 : INFO : PROGRESS: at sentence #830000, processed 5986889 words, keeping 167834 word types
2021-11-29 11:56:51,804 : INFO : PROGRESS: at sentence #840000, processed 6091431 words, keeping 170783 word types
2021-11-29 11:56:51,841 : INFO : PROGRESS: at sentence #850000, processed 6187964 words, keeping 172114 word types
2021-11-29 11:56:51,875 : INFO : PROGRESS: at sentence #860000, processed 6268438 words, keeping 173751 word types
2021-11-29 11:56:51,898 : INFO : collected 174111 word types from a corpus of 6320314 raw words and 865656 sentences
2021-11-29 11:56:51,899 : INFO : Creating a fresh vocabulary
2021-11-29 11:56:

2021-11-29 11:57:55,902 : INFO : EPOCH 1 - PROGRESS: at 92.47% examples, 112362 words/s, in_qsize 39, out_qsize 0
2021-11-29 11:57:56,960 : INFO : EPOCH 1 - PROGRESS: at 93.74% examples, 112275 words/s, in_qsize 39, out_qsize 0
2021-11-29 11:57:58,111 : INFO : EPOCH 1 - PROGRESS: at 94.97% examples, 111410 words/s, in_qsize 39, out_qsize 0
2021-11-29 11:57:58,238 : DEBUG : job loop exiting, total 633 jobs
2021-11-29 11:57:59,120 : INFO : EPOCH 1 - PROGRESS: at 96.98% examples, 112526 words/s, in_qsize 24, out_qsize 0
2021-11-29 11:57:59,992 : DEBUG : worker exiting, processed 35 jobs
2021-11-29 11:57:59,992 : INFO : worker thread finished; awaiting finish of 19 more threads
2021-11-29 11:58:00,119 : DEBUG : worker exiting, processed 30 jobs
2021-11-29 11:58:00,119 : INFO : worker thread finished; awaiting finish of 18 more threads
2021-11-29 11:58:00,136 : DEBUG : worker exiting, processed 31 jobs
2021-11-29 11:58:00,137 : INFO : EPOCH 1 - PROGRESS: at 97.96% examples, 111584 words/s, 

2021-11-29 11:58:43,022 : INFO : EPOCH 2 - PROGRESS: at 85.44% examples, 113057 words/s, in_qsize 39, out_qsize 1
2021-11-29 11:58:44,095 : INFO : EPOCH 2 - PROGRESS: at 87.51% examples, 113814 words/s, in_qsize 39, out_qsize 0
2021-11-29 11:58:45,114 : INFO : EPOCH 2 - PROGRESS: at 88.79% examples, 113369 words/s, in_qsize 39, out_qsize 0
2021-11-29 11:58:46,227 : INFO : EPOCH 2 - PROGRESS: at 90.01% examples, 113115 words/s, in_qsize 39, out_qsize 0
2021-11-29 11:58:47,317 : INFO : EPOCH 2 - PROGRESS: at 91.09% examples, 112734 words/s, in_qsize 39, out_qsize 0
2021-11-29 11:58:48,527 : INFO : EPOCH 2 - PROGRESS: at 92.47% examples, 112690 words/s, in_qsize 39, out_qsize 0
2021-11-29 11:58:49,548 : INFO : EPOCH 2 - PROGRESS: at 94.34% examples, 113463 words/s, in_qsize 38, out_qsize 1
2021-11-29 11:58:50,604 : INFO : EPOCH 2 - PROGRESS: at 95.07% examples, 112215 words/s, in_qsize 39, out_qsize 0
2021-11-29 11:58:50,749 : DEBUG : job loop exiting, total 633 jobs
2021-11-29 11:58:51,7

2021-11-29 11:59:29,599 : INFO : EPOCH 3 - PROGRESS: at 73.27% examples, 114327 words/s, in_qsize 39, out_qsize 0
2021-11-29 11:59:30,669 : INFO : EPOCH 3 - PROGRESS: at 77.35% examples, 114430 words/s, in_qsize 38, out_qsize 1
2021-11-29 11:59:31,722 : INFO : EPOCH 3 - PROGRESS: at 78.72% examples, 114525 words/s, in_qsize 39, out_qsize 0
2021-11-29 11:59:32,888 : INFO : EPOCH 3 - PROGRESS: at 80.24% examples, 114319 words/s, in_qsize 39, out_qsize 0
2021-11-29 11:59:33,917 : INFO : EPOCH 3 - PROGRESS: at 83.28% examples, 114970 words/s, in_qsize 37, out_qsize 2
2021-11-29 11:59:34,935 : INFO : EPOCH 3 - PROGRESS: at 86.06% examples, 115170 words/s, in_qsize 39, out_qsize 0
2021-11-29 11:59:36,171 : INFO : EPOCH 3 - PROGRESS: at 87.58% examples, 114769 words/s, in_qsize 39, out_qsize 0
2021-11-29 11:59:37,215 : INFO : EPOCH 3 - PROGRESS: at 88.89% examples, 114448 words/s, in_qsize 39, out_qsize 0
2021-11-29 11:59:38,269 : INFO : EPOCH 3 - PROGRESS: at 90.01% examples, 114101 words/s,

In [ ]:
train_FastText_model(sub_word = True)

In [ ]:
train_w2v_model(sub_word = False)

In [ ]:
train_w2v_model(sub_word = True)

In [ ]:
import gensim
#w2v with and without subword
sw_W2V_CBOW = gensim.models.Word2Vec.load("./emb_models/subword_wol_W2V_CBOW_50D")
sw_W2V_SG = gensim.models.Word2Vec.load("./emb_models/subword_wol_W2V_Skip-gram_50D")
W2V_CBOW = gensim.models.Word2Vec.load("./emb_models/wol_W2V_CBOW_50D")
W2V_SG = gensim.models.Word2Vec.load("./emb_models/wol_W2V_Skip-gram_50D")

#FastText with and without subword

FT_CBOW = gensim.models.Word2Vec.load("./emb_models/wol_FastText_CBOW_50D")
FT_SG = gensim.models.Word2Vec.load("./emb_models/wol_FastText_Skip-gram_50D")
sw_FT_CBOW = gensim.models.Word2Vec.load("./emb_models/subword_wol_FastText_CBOW_50D")
sw_FT_SG = gensim.models.Word2Vec.load("./emb_models/subword_wol_FastText_Skip-gram_50D")

#print(sw_W2V_CBOW)
#print(sw_FT_CBOW)
#sw_FT_CBOW.wv.vocab



In [ ]:
W2V_SG.wv.most_similar("kettaa")  # 

In [ ]:
sw_FT_CBOW.wv.most_similar("kettaa")  # 

In [ ]:
#rock_idx = sw_FT_CBOW.wv.key_to_index["lo''o"]
#rock_cnt = sw_FT_CBOW.wv.get_vecattr("xossa", "count")  #
vocab_len = len(FT_CBOW.wv)  #
print(vocab_len)

In [ ]:
W2V_SG.wv.similar_by_word("lo\'o")  # 

In [ ]:
data = open("./wol-WordSim-100.txt", "r", encoding="utf8")
for d in data:
    words = d.split()
    print(words[0], words[1], W2V_CBOW.wv.wmdistance(words[0], words[1]))
#W2V_CBOW.wv.similar_by_vector("oduwa")  # 


In [ ]:
data = open("./wol-WordSim-100.txt", "r", encoding="utf8")
for d in data:
    words = d.split()
    print(words[0], words[1], FT_CBOW.wv.similarity(words[0], words[1]))
#W2V_CBOW.wv.similar_by_vector("oduwa")  # 

In [ ]:
# Cosine Similarity for Vector Space Models
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):    
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

In [ ]:
list1 = []
list2 = []
data = open("./wol-WordSim-100.txt", "r", encoding="utf8")
for d in data:
    words = d.split()
    a = FT_CBOW.wv[words[0]]
    #print(a)
    b = FT_CBOW.wv[words[1]]
    #print(b)
    list1.append(words[2])
    c = get_cosine_similarity(a, b)
    list2.append(c)
    #print (words[0], words[1], words[2], c)
print("list1", list1)
print("list2", list2)
#W2V_CBOW.wv.similar_by_vector("oduwa")  # 


In [ ]:
from scipy import stats
stats.spearmanr(list1, list2)

# Spearman Correlation Coefficient
Spearman rank correlation: 
Spearman rank correlation is a non-parametric test 
that is used to measure the degree of association between two variables. 
Like other correlation coefficients, this one varies between -1 and +1 with 0 implying no correlation. 
Correlations of -1 or +1 imply an exact monotonic relationship. 
Positive correlations imply that as x increases, so does y. 
Negative correlations imply that as x increases, y decreases.

In [ ]:
# calculate the spearman's correlation between two variables
from numpy.random import rand
from numpy.random import seed
from scipy.stats import spearmanr
# calculate spearman's correlation
coef, p = spearmanr(list1, list2)
print('Spearmans correlation coefficient: %.3f' % coef)
# interpret the significance
alpha = 0.5
if p > alpha:
    print('Samples are uncorrelated')
else:
    print('Samples are correlated')

In [ ]:
W2V_SG.build_vocab(sentences=common_texts)